In [46]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate

# New Section

In [47]:
!pip install fairlearn

In [48]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate

In [49]:
# Load the dataset
data = pd.read_csv("StudentsPerformance.csv")

In [50]:
# Inspect the data
display(data.head())
print(data.columns)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')


In [51]:
# Preprocess
data = data[['gender', 'race/ethnicity', 'math score', 'reading score', 'writing score']]
data = pd.get_dummies(data, drop_first=True)

In [52]:
# Define a target variable: 'high_math_score' (above median)
median_math_score = data['math score'].median()
data['high_math_score'] = (data['math score'] > median_math_score).astype(int)

# Prepare X and y
X = data.drop(['math score', 'reading score', 'writing score', 'high_math_score'], axis=1)
y = data['high_math_score']
sensitive_feature = X['gender_male'] # Assuming gender is the sensitive feature

In [53]:
# Split data
X_train, X_test, y_train, y_test, sf_train, sf_test = train_test_split(
    X, y, sensitive_feature, test_size=0.3, random_state=42
)

In [54]:
# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [55]:
# Evaluate fairness and accuracy
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.metrics import accuracy_score

metric_frame = MetricFrame(
    metrics={
        "Selection Rate": selection_rate,
        "Accuracy": accuracy_score
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sf_test
)

print("Fair Model Evaluation by Gender:\n")
print(metric_frame.by_group)
print("\nOverall Accuracy:", accuracy_score(y_test, y_pred))

Fair Model Evaluation by Gender:

             Selection Rate  Accuracy
gender_male                          
False              0.090909  0.629371
True               0.694268  0.592357

Overall Accuracy: 0.61
